In [5]:
import openai
import os
import random

openai.api_key = os.environ["OPENAI"] 

In [44]:
sudoku_str = """_2__3819_
_5__7__86
_8791_2__
5__7__62_
_____5_18
643___9__
__94_1_3_
1748_____
__569_8_1""".split("\n")

row_indices = "abcdefghi"
col_indices = "БВГДЕЖЗИК"
square_indices = "αβγδεζηθι"

new_sudoku_str = []
for row in range(9):
    new_row = ""
    for col in range(9): 
        new_row += f"{row_indices[row]}{col_indices[col]}{square_indices[(row//3)*3 + col//3]}: {sudoku_str[row][col]} | "
    new_row = "".join(new_row)
    new_sudoku_str.append(new_row)
    if row in [2, 5]:
        new_sudoku_str.append("=======")
    else:
        new_sudoku_str.append("-------")

grid = "\n".join(new_sudoku_str)
print(grid)

aБα: _ | aВα: 2 | aГα: _ | aДβ: _ | aЕβ: 3 | aЖβ: 8 | aЗγ: 1 | aИγ: 9 | aКγ: _ | 
-------
bБα: _ | bВα: 5 | bГα: _ | bДβ: _ | bЕβ: 7 | bЖβ: _ | bЗγ: _ | bИγ: 8 | bКγ: 6 | 
-------
cБα: _ | cВα: 8 | cГα: 7 | cДβ: 9 | cЕβ: 1 | cЖβ: _ | cЗγ: 2 | cИγ: _ | cКγ: _ | 
dБδ: 5 | dВδ: _ | dГδ: _ | dДε: 7 | dЕε: _ | dЖε: _ | dЗζ: 6 | dИζ: 2 | dКζ: _ | 
-------
eБδ: _ | eВδ: _ | eГδ: _ | eДε: _ | eЕε: _ | eЖε: 5 | eЗζ: _ | eИζ: 1 | eКζ: 8 | 
-------
fБδ: 6 | fВδ: 4 | fГδ: 3 | fДε: _ | fЕε: _ | fЖε: _ | fЗζ: 9 | fИζ: _ | fКζ: _ | 
gБη: _ | gВη: _ | gГη: 9 | gДθ: 4 | gЕθ: _ | gЖθ: 1 | gЗι: _ | gИι: 3 | gКι: _ | 
-------
hБη: 1 | hВη: 7 | hГη: 4 | hДθ: 8 | hЕθ: _ | hЖθ: _ | hЗι: _ | hИι: _ | hКι: _ | 
-------
iБη: _ | iВη: _ | iГη: 5 | iДθ: 6 | iЕθ: 9 | iЖθ: _ | iЗι: 8 | iИι: _ | iКι: 1 | 
-------


In [ ]:

system_prompt = """You are Levy Rosenstein, former mathematician, current sudoku master. 
You are legendary at number puzzles and crosswords. 
You are extremely good in using set theory to solve puzzles. 
You are methodical in your thinking, you never make mistakes, and you can multiply 7 digit numbers in your mind. 
 
Now, you are in a sudoku championship, and you're playing for the grand prize. 
"""

def generate_user_prompt(grid):
    groups = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'К', 'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι']
    sampled_elements = random.sample(groups, 8)
    str_groups = ", ".join(sampled_elements)
    
    user_prompt = f"""All you need to do to win is solve the following sudoku puzzle:

{grid}
You've added indices to each square, indicating the row, the column, and the 3x3 square each square belongs to.
Rows have an index from a, b, c, d, e, f, g, h, i, columns have an index from Б, В, Г, Д, Е, Ж, З, И, К, 3x3 squares have an index from α, β, γ, δ, ε, ζ, η, θ, ι. Empty spaces are represented as "_".
You call each column, row, and 3x3 square a "group". Each group should have the numbers from 1 to 9 exactly. Every group has exactly 9 squares.  A group CANNOT contain the same number twice. A group MUST contain every number from 1 to 9.

Using this index, you can tell exactly to which groups a square belongs.
Example: gБη belongs to g, Б, η. 
Example: dЕε belongs to d, Е, and ε. 

You can also tell which squares belong in a certain group: all who contain the group in their index.
Example: group 'a' contains squares aБα, aВα, aГα, aДβ, aЕβ, aЖβ, aЗγ, aИγ, aКγ. Therefore, group 'a' contains the numbers ...
Example: group К contains squares aКγ, bКγ, cКγ, dКζ, eКζ, fКζ, gКι, hКι, iКι. Therefore, group К contains the numbers ...

Additionally: groups intersect. The squares which have the indices of two groups, belong to both groups.
Example: group a intersects group α in the squares aБα, aВα, aГα.
Example: column Е intersects row f only in the square fЕε.

Please solve it with the following steps:

1) For these random groups: {str_groups}, name the numbers they contain, then the numbers remaining, then the number filled in squares and the number of empty squares. Be very concise. Do this until you have named 6 groups, alternating between column, row, 3x3 square.
Example:
"α - contains 1, 7, 4, 3, 8; missing - 2, 5, 6, 9; 5 filled in, 4 missing."
Example:
"З - contains 2, 3, 4, 5, 8, 9; missing - 1, 6, 7; 6 filled in, 3 missing."

2) Pick and target one specific group from the ones you've chosen, and lets see if we can deduce anything about it. We will call it the "target group". Which group out of these seems most promising?  
Name all of it's empty squares, and the numbers we need to fill them with. Name all groups that intersect with its empty squares.

Example:
"Group g seems the most promising, because it has 7 numbers filled in, and only 2 empty squares. It contains the empty squares ... and ... and intersects groups .. , .. and .."

4) Now that we have the missing numbers from our target group, and the intersecting groups, we can make deductions.
Specifically, the know what numbers we need to fill in in the empty squares of our target group. But we also know the numbers that appear in the intersecting groups, and we can therefore exclude some possibilities for some squares.

Make some observations about the target group and the intersecting groups.

The goal is to be able to do the following:
4.1) in a given group (row/column/3x3square) exclude possibilities for a given square until it can only be one number
4.2) in a given group (row/column/3x3square) exclude possibilities for a the position of a given number until it can only be in one square

Example: 
"We need to fill in numbers 1, 2, 7 in squares dВδ, eБδ, eГδ. 
Intersecting group В contains 1 and 2, therefore square dВδ cannot be 1 or 2. 
Intersecting group e contains 7, therefore squares eБδ and eГδ cannot be 7.
Intersecting group d does not contain 1, 2 or 7, therefore we can't use it."

5) Combine all your observations. For each empty square in the target group, what are the possible numbers that remain? Can you declare for certain what number is in an empty square in your target group? Say so. 

Example: 
Group X has 3 empty squares that must be filled in with the numbers 1, 2, 5. But intersecting group Z contains 5, and overlaps two of the empty squares, therefore 5 can only be placed in the last empty square.

Example:
Group Y has 4 empty squares, that must be filled in with the numbers 1, 2, 3, 4. But intersecting groups Q and P both contain 1, therefore 1 must be placed in the last empty square that doesn't intersect Q and P.

Example:
Row X has 3 empty squares, which must be filled in with the numbers 5, 7, 9. However, intersecting column Y contains 5 and 7, therefore the empty square which belongs to both X and Y must be exactly 9. 

Important - not always will you be able fill in a number in your target group. That is fine, if so, return to step 1 and choose a different group.
If there is only one possibility for any of the empty squares, fill it in and proceed to step 6. 

6) If you have deduced for certain what an empty square must contain, fill it in.
Reproduce the sudoku grid with the new number filled in, in exactly the same format, including coordinates.
"""
    return user_prompt

In [10]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0,
)

In [43]:
answers = []
grids = [grid]
user_prompt = generate_user_prompt(grid)

for step in range(5):
    print(f"now on step {step}")
    
    response = response = openai.ChatCompletion.create(
                  model="gpt-4",
                  messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt},
                    ],
                    temperature=0,
                )
    
    answer = response['choices'][0]['message']['content']
    answers.append(answer)
    grid_start = answer.rfind("aБα: ")
    new_grid = answer[grid_start:]
    
    print(new_grid)
    
    # check if it's a valid grid, or the process failed on step 5
    corners = ["aБα", "iБη", "aКγ", "iКι"]
    if all(corner in new_grid for corner in corners):
        user_prompt = generate_user_prompt(new_grid)
        answers.append(answer)
        grids.append(new_grid)
        print("successfully added another number!\n")
    else:
        print("did not find a number, trying again ... \n")

now on step 0
aБα: 9 | aВα: 2 | aГα: 5 | aДβ: _ | aЕβ: 3 | aЖβ: 8 | aЗγ: 1 | aИγ: 9 | aКγ: _ | 
-------
bБα: _ | bВα: 5 | bГα: _ | bДβ: _ | bЕβ: 7 | bЖβ: _ | bЗγ: _ | bИγ: 8 | bКγ: 6 | 
-------
cБα: _ | cВα: 8 | cГα: 7 | cДβ: 9 | cЕβ: 1 | cЖβ: _ | cЗγ: 2 | cИγ: _ | cКγ: _ | 
dБδ: 5 | dВδ: _ | dГδ: _ | dДε: 7 | dЕε: _ | dЖε: _ | dЗζ: 6 | dИζ: 2 | dКζ: _ | 
-------
eБδ: _ | eВδ: _ | eГδ: _ | eДε: _ | eЕε: _ | eЖε: 5 | eЗζ: _ | eИζ: 1 | eКζ: 8 | 
-------
fБδ: 6 | fВδ: 4 | fГδ: 3 | fДε: _ | fЕε: _ | fЖε: _ | fЗζ: 9 | fИζ: _ | fКζ: _ | 
gБη: _ | gВη: _ | gГη: 9 | gДθ: 4 | gЕθ: _ | gЖθ: 1 | gЗι: _ | gИι: 3 | gКι: _ | 
-------
hБη: 1 | hВη: 7 | hГη: 4 | hДθ: 8 | hЕθ: _ | hЖθ: _ | hЗι: _ | hИι: _ | hКι: _ | 
-------
iБη: _ | iВη: _ | iГη: 5 | iДθ: 6 | iЕθ: 9 | iЖθ: _ | iЗι: 8 | iИι: _ | iКι: 1 |
successfully added another number!

now on step 1
.
problem4e...

now on step 2
aБα: 9 | aВα: 2 | aГα: 5 | aДβ: 4 | aЕβ: 3 | aЖβ: 8 | aЗγ: 1 | aИγ: 9 | aКγ: 3 | 
-------
bБα: 6 | bВα: 5 | bГα: 6 | bДβ: